In [1]:
# === 필수 라이브러리 임포트 ===
import pandas as pd
import numpy as np
import os
import gc
import time
from tqdm.auto import tqdm
tqdm.pandas()

# 머신러닝 모델 및 도구
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score

# 딥러닝 및 임베딩
import torch
from transformers import AutoTokenizer, AutoModel

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [2]:
# === 기본 데이터 로드 ===
print("데이터를 로드합니다...")
train_df_original = pd.read_csv('data/train.csv', encoding='utf-8-sig')
test_df = pd.read_csv('data/test.csv', encoding='utf-8-sig')
sample_submission = pd.read_csv('data/sample_submission.csv', encoding='utf-8-sig')

# === 문단 분리 ===
print("문단 분리된 데이터프레임을 생성합니다...")
train_df_original['paragraphs'] = train_df_original['full_text'].str.split('\n')
train_paragraph_df = train_df_original.explode('paragraphs').rename(columns={'paragraphs': 'text'})
train_paragraph_df.dropna(subset=['text'], inplace=True)
train_paragraph_df = train_paragraph_df[train_paragraph_df['text'].str.strip().astype(bool)].reset_index(drop=True)

# === 미리 생성된 기본 특징 로드 ===
print("미리 생성된 특징(.npy) 파일을 로드합니다...")
X_train_paragraph_features = np.load('data/X_train_paragraph_features.npy')
X_test_paragraph_features = np.load('data/X_test_paragraph_features.npy')

X_train_final = np.load('X_train_final.npy')
X_test_final = np.load('X_test_final.npy')

print(f"훈련 데이터: {train_paragraph_df.shape}")
print(f"기존 훈련 특징: {X_train_paragraph_features.shape}")
print("✅ 데이터 로드 및 준비 완료")

데이터를 로드합니다...
문단 분리된 데이터프레임을 생성합니다...
미리 생성된 특징(.npy) 파일을 로드합니다...
훈련 데이터: (1226364, 4)
기존 훈련 특징: (1226364, 1664)
✅ 데이터 로드 및 준비 완료


In [3]:
# print("--- 1단계 (업그레이드): 문체/통계적 특징 생성 시작 ---")

# # 한국어 불용어 리스트 (필요에 따라 추가/삭제 가능)
# STOPWORDS = set(['은', '는', '이', '가', '을', '를', '의', '에', '와', '과', '도', '으로', '로',
#                  '하다', '이다', '있다', '그', '저', '이것', '저것', '그것', '및', '등'])
# if 'paragraph_text' in test_df.columns:
#     print("test_df의 'paragraph_text' 컬럼을 'text'로 변경합니다.")
#     test_df.rename(columns={'paragraph_text': 'text'}, inplace=True)

# def create_statistical_features(df):
#     """더 의미있는 통계/문체적 특징을 생성합니다."""
#     # 기본 개수 특징
#     df['text_len'] = df['text'].str.len()
#     df['word_count'] = df['text'].str.split().str.len()
#     df['sentence_count'] = df['text'].str.count(r'[.!?]') + 1
#     df['comma_count'] = df['text'].str.count(',')
#     # SyntaxWarning 해결 (r 추가)
#     df['special_char_count'] = df['text'].str.count(r'[^A-Za-z0-9가-힣\s]')
#     df['lexical_diversity'] = df['text'].apply(lambda x: len(set(x.split())) / (len(x.split()) + 1e-6))
    
#     # [새로운 특징] 비율 특징 (0으로 나누는 것을 방지하기 위해 1e-6 추가)
#     df['avg_sentence_len'] = df['word_count'] / df['sentence_count']
#     df['avg_word_len'] = df['text_len'] / (df['word_count'] + 1e-6)
    
#     # [새로운 특징] 불용어 비율
#     df['stopword_count'] = df['text'].apply(lambda x: sum(word in STOPWORDS for word in x.split()))
#     df['stopword_ratio'] = df['stopword_count'] / (df['word_count'] + 1e-6)
    
#     # 결측치(NaN)가 발생할 경우 0으로 채움
#     df.fillna(0, inplace=True)
    
#     return df

# # 훈련 및 테스트 데이터에 새로운 함수 적용
# train_paragraph_df = create_statistical_features(train_paragraph_df)
# test_df = create_statistical_features(test_df)

# # 사용할 특징 이름 리스트 업데이트
# new_stat_feature_names = [
#     'text_len', 'word_count', 'sentence_count', 'comma_count', 'special_char_count', 'lexical_diversity',
#     'avg_sentence_len', 'avg_word_len', 'stopword_ratio' # stopword_count는 비율 계산에만 사용
# ]
# train_stat_features = train_paragraph_df[new_stat_feature_names].values
# test_stat_features = test_df[new_stat_feature_names].values

# # 기존 특징과 새로 만든 통계 특징을 결합
# X_train_combined = np.c_[X_train_paragraph_features, train_stat_features]
# X_test_combined = np.c_[X_test_paragraph_features, test_stat_features]

# print("✅ 업그레이드된 통계 특징 생성 및 결합 완료!")
# print(f"1단계 후 훈련 특징 shape: {X_train_combined.shape}")

In [4]:
# print("--- 2단계: KoELECTRA 임베딩 생성 시작 ---")

# # GPU 설정
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# model_name = "monologg/koelectra-base-v3-discriminator"
# koelectra_tokenizer = AutoTokenizer.from_pretrained(model_name)

# # SafeTensors 사용 및 RTX 3060 최적화
# try:
#     koelectra_model = AutoModel.from_pretrained(
#         model_name,
#         use_safetensors=True,  # 보안 강화
#         torch_dtype=torch.float16,  # 메모리 효율성 향상
#         device_map="auto"  # 자동 메모리 관리
#     ).to(device)
#     print("✅ SafeTensors 형식으로 모델 로드 성공")
# except Exception as e:
#     print(f"⚠️ SafeTensors 로드 실패, 기본 방식으로 시도: {e}")
#     koelectra_model = AutoModel.from_pretrained(model_name).half().to(device)

# def get_embeddings(texts, model, tokenizer, batch_size=128):  # RTX 3060 최적화: 256→128
#     """주어진 모델과 토크나이저로 텍스트 임베딩을 추출합니다."""
#     all_embeddings = []
#     model.eval()
    
#     for i in tqdm(range(0, len(texts), batch_size), desc="Extracting Embeddings"):
#         batch = texts[i:i+batch_size]
#         batch_dict = tokenizer(
#             [str(t) for t in batch], 
#             max_length=256, 
#             padding=True, 
#             truncation=True, 
#             return_tensors='pt'
#         ).to(device)
        
#         with torch.no_grad(), torch.cuda.amp.autocast():
#             outputs = model(**batch_dict)
        
#         embeddings = outputs.last_hidden_state[:, 0, :].float().cpu().numpy()
#         all_embeddings.append(embeddings)
        
#         # 메모리 정리 (10 배치마다)
#         if i % 10 == 0:
#             torch.cuda.empty_cache()
    
#     return np.vstack(all_embeddings)

# # GPU 메모리 상태 확인
# def check_gpu_memory():
#     if torch.cuda.is_available():
#         print(f"GPU Memory - Allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f}GB, "
#               f"Reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f}GB")

# print("모델 로딩 후 GPU 메모리 상태:")
# check_gpu_memory()

# # KoELECTRA 임베딩 생성
# print("훈련 데이터 임베딩 생성 중...")
# ko_electra_train_features = get_embeddings(train_paragraph_df['text'].tolist(), koelectra_model, koelectra_tokenizer)

# print("테스트 데이터 임베딩 생성 중...")
# ko_electra_test_features = get_embeddings(test_df['text'].tolist(), koelectra_model, koelectra_tokenizer)

# # GPU 메모리 정리
# del koelectra_model
# gc.collect()
# torch.cuda.empty_cache()

# print("메모리 정리 후 GPU 상태:")
# check_gpu_memory()

# # 1단계에서 만든 특징에 KoELECTRA 임베딩을 최종적으로 결합
# X_train_final = np.c_[X_train_combined, ko_electra_train_features]
# X_test_final = np.c_[X_test_combined, ko_electra_test_features]

# # 나중을 위해 최종 특징 파일을 저장해두는 것이 안전합니다.
# print("최종 특징 파일을 저장합니다...")
# np.save('X_train_final.npy', X_train_final)
# np.save('X_test_final.npy', X_test_final)

# print("✅ KoELECTRA 임베딩 결합 및 최종 특징 생성 완료!")
# print(f"최종 훈련 특징 shape: {X_train_final.shape}")
# print(f"최종 테스트 특징 shape: {X_test_final.shape}")


In [5]:
print("--- 3단계: 의사 라벨링으로 학습 데이터 정제 시작 ---")

# 1. 1차 모델(LGBM)을 Noisy Label로 빠르게 학습
print("1차 모델을 Noisy Label로 학습합니다...")
lgb_pseudo = lgb.LGBMClassifier(objective='binary', metric='auc', random_state=42, n_jobs=-1)
noisy_labels = train_paragraph_df['generated'].values
lgb_pseudo.fit(X_train_final, noisy_labels)

# 2. 모든 훈련 데이터에 대해 AI일 확률 예측
print("모든 훈련 문단에 대해 AI일 확률을 예측합니다...")
train_preds_proba = lgb_pseudo.predict_proba(X_train_final)[:, 1]

# 3. 신뢰도 높은 데이터만 선별하여 최종 훈련 세트 구성
print("신뢰도 높은 데이터만 선별하여 최종 훈련 세트를 구성합니다...")
ai_doc_indices = train_paragraph_df.index[train_paragraph_df['generated'] == 1]
human_doc_indices = train_paragraph_df.index[train_paragraph_df['generated'] == 0]

# AI 글 중에서, 1차 모델이 AI일 확률이 0.9 이상이라고 매우 강하게 예측한 문단만 선택
CONFIDENCE_THRESHOLD = 0.9
confident_ai_indices = ai_doc_indices[train_preds_proba[ai_doc_indices] > CONFIDENCE_THRESHOLD]

# 최종 훈련에 사용할 인덱스: (확실한 사람 글) + (매우 확실한 AI 글)
final_clean_indices = np.concatenate([human_doc_indices, confident_ai_indices])
np.random.shuffle(final_clean_indices)

# 최종적으로 정제된 훈련 데이터 생성
X_train_clean = X_train_final[final_clean_indices]
y_train_clean = train_paragraph_df['generated'].iloc[final_clean_indices].values
groups_clean = train_paragraph_df['title'].iloc[final_clean_indices].values

print(f"✅ 데이터 정제 완료!")
print(f"원본 훈련 데이터 수: {len(X_train_final):,}")
print(f"정제된 훈련 데이터 수: {len(X_train_clean):,}")
print(f"정제된 AI 문단 수: {(y_train_clean == 1).sum():,}")

--- 3단계: 의사 라벨링으로 학습 데이터 정제 시작 ---
1차 모델을 Noisy Label로 학습합니다...
[LightGBM] [Info] Number of positive: 100712, number of negative: 1125652
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 18.613746 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 613406
[LightGBM] [Info] Number of data points in the train set: 1226364, number of used features: 2441
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082122 -> initscore=-2.413853
[LightGBM] [Info] Start training from score -2.413853
모든 훈련 문단에 대해 AI일 확률을 예측합니다...


d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


신뢰도 높은 데이터만 선별하여 최종 훈련 세트를 구성합니다...
✅ 데이터 정제 완료!
원본 훈련 데이터 수: 1,226,364
정제된 훈련 데이터 수: 1,153,741
정제된 AI 문단 수: 28,089


In [6]:
print("--- 4-1단계: LightGBM 모델 학습 시작 (특징 선택 포함) ---")

# --- 메모리 절약을 위한 데이터 타입 변경 ---
print("데이터 타입을 float32로 변경하여 메모리 부담을 줄입니다...")
X_train_clean = X_train_clean.astype('float32')
X_test_final = X_test_final.astype('float32')
gc.collect()

# --- 특징 선택(Feature Selection) 시작 ---
print("\n특징 선택을 시작합니다...")

# 1. 중요도 측정을 위해 데이터의 일부 샘플(20만개)만 사용
SAMPLE_SIZE = 200000
if len(X_train_clean) > SAMPLE_SIZE:
    sample_indices = np.random.choice(len(X_train_clean), SAMPLE_SIZE, replace=False)
    X_train_sample = X_train_clean[sample_indices]
    y_train_sample = y_train_clean[sample_indices]
else:
    X_train_sample, y_train_sample = X_train_clean, y_train_clean

# 2. 샘플 데이터로 단일 LGBM 모델을 빠르게 학습
print(f"{len(X_train_sample):,}개의 샘플로 특징 중요도 측정 모델을 학습합니다...")
fs_model = lgb.LGBMClassifier(objective='binary', metric='auc', n_estimators=500, n_jobs=16, random_state=42)
fs_model.fit(X_train_sample, y_train_sample)

# 3. 중요도가 높은 상위 800개 특징의 인덱스를 선택
NUM_TOP_FEATURES = 800
feature_importances = fs_model.feature_importances_
top_feature_indices = np.argsort(feature_importances)[-NUM_TOP_FEATURES:]

# 4. 선택된 특징으로 새로운 '슬림' 데이터셋 생성
print(f"중요도 상위 {NUM_TOP_FEATURES}개의 특징만 선택하여 새로운 데이터셋을 생성합니다.")
X_train_slim = X_train_clean[:, top_feature_indices]
X_test_slim = X_test_final[:, top_feature_indices]

print(f"특징 선택 완료! 새로운 훈련 데이터 shape: {X_train_slim.shape}")

# 메모리 정리
del X_train_sample, y_train_sample, fs_model, feature_importances
gc.collect()


# --- 선택된 특징(X_train_slim)으로 5-Fold 교차검증 진행 ---
print("\n선택된 특징으로 최종 모델 학습을 시작합니다...")
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
lgb_oof_preds = np.zeros(len(X_train_slim))
lgb_test_preds_list = []
lgb_models = []

lgb_params = {'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
              'n_estimators': 2000, 'learning_rate': 0.05, 'num_leaves': 41,
              'max_depth': 6, 'seed': 42, 'n_jobs': 16, 'verbose': -1, 'colsample_bytree': 0.8}

# groups_clean도 slim 데이터셋에 맞게 인덱싱해야 합니다. (이 부분은 동일)
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_slim, y_train_clean, groups_clean)):
    print(f"--- Fold {fold+1} (LGBM) ---")
    X_train, y_train = X_train_slim[train_idx], y_train_clean[train_idx]
    X_val, y_val = X_train_slim[val_idx], y_train_clean[val_idx]
    
    lgb_model = lgb.LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(100, verbose=False)])
    
    lgb_oof_preds[val_idx] = lgb_model.predict_proba(X_val)[:, 1]
    # 테스트 데이터 예측도 slim 버전으로 해야 함
    lgb_test_preds_list.append(lgb_model.predict_proba(X_test_slim)[:, 1])
    lgb_models.append(lgb_model)
    
    del X_train, y_train, X_val, y_val, lgb_model
    gc.collect()

lgb_oof_auc = roc_auc_score(y_train_clean, lgb_oof_preds)
print(f"🏆 LightGBM 최종 OOF AUC: {lgb_oof_auc:.5f}")

--- 4-1단계: LightGBM 모델 학습 시작 (특징 선택 포함) ---
데이터 타입을 float32로 변경하여 메모리 부담을 줄입니다...

특징 선택을 시작합니다...
200,000개의 샘플로 특징 중요도 측정 모델을 학습합니다...
[LightGBM] [Info] Number of positive: 4899, number of negative: 195101
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 3.764241 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 613432
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 2441
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.024495 -> initscore=-3.684486
[LightGBM] [Info] Start training from score -3.684486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


--- Fold 2 (LGBM) ---


d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


--- Fold 3 (LGBM) ---


...

--- Fold 4 (LGBM) ---


d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


--- Fold 5 (LGBM) ---


d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
d:\develop\deep_learning\DACON\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🏆 LightGBM 최종 OOF AUC: 0.99886


In [8]:
print("--- 4-2단계: XGBoost 모델 학습 시작 ---")

xgb_oof_preds = np.zeros(len(X_train_clean))
xgb_test_preds_list = []
xgb_models = []

xgb_params = {'objective': 'binary:logistic', 'eval_metric': 'auc', 'eta': 0.05,
              'max_depth': 5, 'subsample': 0.7, 'colsample_bytree': 0.8,
              'device': 'cuda', 'tree_method': 'hist', 'random_state': 42}

for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_clean, y_train_clean, groups_clean)):
    print(f"--- Fold {fold+1} (XGBoost) ---")
    X_train, y_train = X_train_clean[train_idx], y_train_clean[train_idx]
    X_val, y_val = X_train_clean[val_idx], y_train_clean[val_idx]
    xgb_model = xgb.XGBClassifier(**xgb_params, n_estimators=2000)
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
    xgb_oof_preds[val_idx] = xgb_model.predict_proba(X_val)[:, 1]
    xgb_test_preds_list.append(xgb_model.predict_proba(X_test_final)[:, 1])
    xgb_models.append(xgb_model)

xgb_oof_auc = roc_auc_score(y_train_clean, xgb_oof_preds)
print(f"🏆 XGBoost 최종 OOF AUC: {xgb_oof_auc:.5f}")

--- 4-2단계: XGBoost 모델 학습 시작 ---
--- Fold 1 (XGBoost) ---


KeyboardInterrupt: 

In [9]:
print("--- 5단계: 최종 제출 (LGBM 단일 모델) ---")

# LightGBM 5-Fold 모델들의 테스트 데이터 예측값 평균 계산
lgb_final_preds = np.mean(lgb_test_preds_list, axis=0)
print("LightGBM 단일 모델의 예측값으로 제출 파일을 생성합니다.")

# 제출 파일 생성
sample_submission['generated'] = lgb_final_preds

# 파일 이름에 lgbm_only를 추가하여 구분
submission_filename = f"submission_lgbm_only_{pd.Timestamp.now().strftime('%Y%m%d_%H%M')}.csv"
sample_submission.to_csv(submission_filename, index=False)
sample_submission.to_csv("submission.csv", index=False) # 기본 파일도 생성

print(f"🎉 최종 제출 파일 생성 완료: {submission_filename}")

--- 5단계: 최종 제출 (LGBM 단일 모델) ---
LightGBM 단일 모델의 예측값으로 제출 파일을 생성합니다.
🎉 최종 제출 파일 생성 완료: submission_lgbm_only_20250715_0916.csv
